In [1]:
from dask_yarn import YarnCluster
from dask.distributed import Client
import os
os.environ['CRYPTOGRAPHY_OPENSSL_NO_LEGACY'] = '1'

/home/hadoop/miniconda/envs/dfs/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import (
/home/hadoop/miniconda/envs/dfs/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: parse_timedelta is deprecated and will be removed in a future release. Please use dask.utils.parse_timedelta instead.
  from distributed.utils import (


In [2]:
cluster = YarnCluster()

2023-09-22 05:24:03,453 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2023-09-22 05:24:03,455 - distributed.scheduler - INFO - State start
2023-09-22 05:24:03,457 - distributed.scheduler - INFO -   Scheduler at: tcp://172.31.36.178:40869
2023-09-22 05:24:03,458 - distributed.scheduler - INFO -   dashboard at:  http://172.31.36.178:44379/status
23/09/22 05:24:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 05:24:04 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-36-178.ap-southeast-1.compute.internal/172.31.36.178:8032
23/09/22 05:24:04 INFO client.AHSProxy: Connecting to Application History server at ip-172-31-36-178.ap-southeast-1.compute.internal/172.31.36.178:10200
23/09/22 05:24:05 INFO skein.Driver: Driver started, listening on 40353
23/

In [3]:
client = Client(cluster.scheduler_address)

2023-09-22 05:24:10,933 - distributed.scheduler - INFO - Receive client connection: Client-429b732b-5908-11ee-8025-06d7787525d8
2023-09-22 05:24:10,935 - distributed.core - INFO - Starting established connection to tcp://172.31.36.178:50520


In [4]:
cluster.scale(1)

In [5]:
cluster.workers()

[Container<service_name='dask.worker', instance=0, state=RUNNING>]

In [6]:
# %%
import os
import featuretools as ft
import numpy as np
import pandas as pd
import dask.dataframe as dd
import time

# %%
data_dir = '/home/hadoop/avs/'
history = dd.read_parquet("s3://dgl-data-private/zymao_dfs/avs/history.parquet/*.parquet", engine='pyarrow', storage_options={'use_ssl': False})
transactions = dd.read_parquet('s3://dgl-data-private/zymao_dfs/avs/transactions.parquet/*.parquet',engine='pyarrow', storage_options={'use_ssl': False})
transactions = transactions.reset_index()
offers = dd.read_parquet('s3://dgl-data-private/zymao_dfs/avs/offer.parquet/*.parquet',engine='pyarrow', storage_options={'use_ssl': False})

# %%
history = history[['chain','offer','market','repeater','offerdate','id']]
transactions = transactions[['index','id', 'chain',   'dept'   ,'category','company', 'brand'  ,'date','productsize','productmeasure','purchasequantity','purchaseamount']]
offers  = offers [[ 'offer' ,   'category',    'quantity' ,     'company' ,   'offervalue','brand'  ]]

# %%
print(history.columns)
print(transactions.columns)
print(offers.columns)

# %%
from woodwork.logical_types import Datetime
dateformat=Datetime(datetime_format="1970-01-01")

es = ft.EntitySet(id='history')
es.add_dataframe(dataframe_name='history',
                                dataframe=history,
                                index='id',
                                logical_types={
                                    'id':'Integer',
                                    'chain': "Categorical",
                                                'market': "Categorical",
                                                'offer': "Integer",
                                                'offerdate': dateformat,
                                                'repeater': "Categorical"
                                                }
                                )
es.add_dataframe(dataframe_name='transactions',
                                dataframe=transactions,
                                index='index',
                                logical_types= 
                                            {
                                                'index':'Integer',
                                                'id':'Integer',
                                                'chain': "Categorical",
                                                'dept': "Categorical",
                                                'category': "Categorical",
                                                'company': "Categorical",
                                                'brand': "Categorical",
                                                'date': dateformat,
                                                'productsize': "Double",
                                                'productmeasure':'Categorical',
                                                'purchasequantity': "Double",
                                                'purchaseamount': "Double"
                                                }
                                )

es.add_dataframe(dataframe_name='offers',
                                dataframe=offers,
                                index='offer',
                                logical_types= 
                                            {
                                                'offer':'Integer',
                                                'category': "Categorical",
                                                'quantity': "Double",
                                                'company': "Categorical",
                                                'offervalue': "Double",
                                                'brand': "Categorical"
                                                }
                                )
# Define relationship
es.add_relationship(
parent_dataframe_name="history",
parent_column_name="id",
child_dataframe_name="transactions",
child_column_name="id")

es.add_relationship(
parent_dataframe_name="offers",
parent_column_name="offer",
child_dataframe_name="history",
child_column_name="offer")
# # %%

# %%
# %%
import warnings

from featuretools.computational_backends import calculate_feature_matrix
from featuretools.entityset import EntitySet
from featuretools.exceptions import UnusedPrimitiveWarning
from featuretools.synthesis.deep_feature_synthesis import DeepFeatureSynthesis
from featuretools.synthesis.utils import _categorize_features, get_unused_primitives
from featuretools.synthesis.dfs import warn_unused_primitives

# %%
dataframes=None
relationships=None
entityset=es
target_dataframe_name="history"
cutoff_time=None
instance_ids=None
agg_primitives=['count','mean','min','max']
# trans_primitives=['weekday']
trans_primitives=[]
groupby_trans_primitives=None
allowed_paths=None
max_depth=2
ignore_dataframes=None
# ignore_dataframes=["pageviews"]
ignore_columns=None
primitive_options=None
seed_features=None
drop_contains=None
drop_exact=None
where_primitives=None
max_features=-1
cutoff_time_in_index=False
save_progress=None
features_only=False
training_window=None
approximate=None
chunk_size=None
n_jobs=1
dask_kwargs=None
verbose=True
return_types=None
progress_callback=None
include_cutoff_time=False

# %%
dfs_object = DeepFeatureSynthesis(
    target_dataframe_name,
    entityset,
    agg_primitives=agg_primitives,
    trans_primitives=trans_primitives,
    groupby_trans_primitives=groupby_trans_primitives,
    max_depth=max_depth,
    where_primitives=where_primitives,
    allowed_paths=allowed_paths,
    drop_exact=drop_exact,
    drop_contains=drop_contains,
    ignore_dataframes=ignore_dataframes,
    ignore_columns=ignore_columns,
    primitive_options=primitive_options,
    max_features=max_features,
    seed_features=seed_features,
)

# %%
features = dfs_object.build_features(verbose=verbose, return_types=return_types)

trans, agg, groupby, where = _categorize_features(features)

trans_unused = get_unused_primitives(trans_primitives, trans)
agg_unused = get_unused_primitives(agg_primitives, agg)
groupby_unused = get_unused_primitives(groupby_trans_primitives, groupby)
where_unused = get_unused_primitives(where_primitives, where)

unused_primitives = [trans_unused, agg_unused, groupby_unused, where_unused]
if any(unused_primitives):
    warn_unused_primitives(unused_primitives)

for feature in features:
    print(f"{feature}")



# %%


Index(['chain', 'offer', 'market', 'repeater', 'offerdate', 'id'], dtype='object')
Index(['index', 'id', 'chain', 'dept', 'category', 'company', 'brand', 'date',
       'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
      dtype='object')
Index(['offer', 'category', 'quantity', 'company', 'offervalue', 'brand'], dtype='object')
Built 30 features
<Feature: chain>
<Feature: offer>
<Feature: market>
<Feature: repeater>
<Feature: COUNT(transactions)>
<Feature: MAX(transactions.productsize)>
<Feature: MAX(transactions.purchaseamount)>
<Feature: MAX(transactions.purchasequantity)>
<Feature: MEAN(transactions.productsize)>
<Feature: MEAN(transactions.purchaseamount)>
<Feature: MEAN(transactions.purchasequantity)>
<Feature: MIN(transactions.productsize)>
<Feature: MIN(transactions.purchaseamount)>
<Feature: MIN(transactions.purchasequantity)>
<Feature: offers.category>
<Feature: offers.quantity>
<Feature: offers.company>
<Feature: offers.offervalue>
<Feature: offers.bra

In [7]:
history.head()

2023-09-22 05:24:34,569 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.42.174:34559', name: dask.worker_0, status: init, memory: 0, processing: 0>
2023-09-22 05:24:34,570 - distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.42.174:34559
2023-09-22 05:24:34,571 - distributed.core - INFO - Starting established connection to tcp://172.31.42.174:45800


RuntimeError: OpenSSL 3.0's legacy provider failed to load. This is a fatal error by default, but cryptography supports running without legacy algorithms by setting the environment variable CRYPTOGRAPHY_OPENSSL_NO_LEGACY. If you did not expect this error, you have likely made a mistake with your OpenSSL configuration.

In [10]:
# %%
feature_matrix = calculate_feature_matrix(
    features,
    entityset=entityset,
    cutoff_time=cutoff_time,
    instance_ids=instance_ids,
    training_window=training_window,
    approximate=approximate,
    cutoff_time_in_index=cutoff_time_in_index,
    save_progress=save_progress,
    chunk_size=chunk_size,
    n_jobs=n_jobs,
    dask_kwargs=dask_kwargs,
    verbose=verbose,
    progress_callback=progress_callback,
    include_cutoff_time=include_cutoff_time
)

RuntimeError: OpenSSL 3.0's legacy provider failed to load. This is a fatal error by default, but cryptography supports running without legacy algorithms by setting the environment variable CRYPTOGRAPHY_OPENSSL_NO_LEGACY. If you did not expect this error, you have likely made a mistake with your OpenSSL configuration.

In [ ]:
start = time.time()
result = feature_matrix.compute()
result.to_parquet("./result.parquet")
end = time.time()
print(f"time: {end-start}")